# PostgreSQL 16.6+

## 0) 前提

* エンジン: **PostgreSQL 16.6+**
* 並び順: 任意
* `NOT IN` 回避（`EXISTS` / `LEFT JOIN ... IS NULL` を推奨）
* 判定は ID 基準、表示は仕様どおり

## 1) 問題

* `会社名 "RED" に紐づく注文を一度も担当していない営業担当者（SalesPerson）の name を求めよ。`
* 入力: `SalesPerson(sales_id, name, salary, commission_rate, hire_date)`, `Company(com_id, name, city)`, `Orders(order_id, order_date, com_id, sales_id, amount)`
* 出力: `name` のみ（任意順）

## 2) 最適解（単一クエリ）

> 本件はウィンドウ不要。**RED の注文に関与した sales_id を抽出 → 反結合**で除外します（NULL 安全）。

```sql
WITH red_company AS (
  SELECT com_id
  FROM Company
  WHERE name = 'RED'
),
red_sales AS (
  SELECT DISTINCT o.sales_id
  FROM Orders o
  JOIN red_company rc ON rc.com_id = o.com_id
)
SELECT s.name
FROM SalesPerson s
LEFT JOIN red_sales r ON r.sales_id = s.sales_id
WHERE r.sales_id IS NULL;

Runtime 413 ms
Beats 17.81%

```

### 代替（`NOT EXISTS` 相関サブクエリ）

```sql
SELECT s.name
FROM SalesPerson s
WHERE NOT EXISTS (
  SELECT 1
  FROM Orders o
  JOIN Company c ON c.com_id = o.com_id
  WHERE c.name = 'RED'
    AND o.sales_id = s.sales_id
);

Runtime 317 ms
Beats 79.25%

```

## 3) 要点解説

* **反結合パターン**（`LEFT JOIN ... IS NULL` / `NOT EXISTS`）は `NOT IN` より **NULL に強い**。
* `red_sales` を **`DISTINCT`** でユニーク化して行数を減らすと結合が軽い。
* 推奨インデックス：`Company(name)`, `Orders(com_id, sales_id)`, `SalesPerson(sales_id)`。

## 4) 計算量（概算）

* `red_sales` 抽出：`Orders` を `com_id` でフィルタ → 索引前提で **O(N)** 近似
* 反結合：`SalesPerson` と `red_sales` の結合 → **O(M)** 近似
* 全体はインデックス最適化で **線形近似**。重複除去 `DISTINCT` は **O(K log K)** 程度（K は RED に関与した注文数）。

## 5) 図解（Mermaid 超保守版）

```mermaid
flowchart TD
  A[Company から RED の com_id] --> B[Orders を com_id で絞り sales_id をユニーク化]
  B --> C[SalesPerson と LEFT JOIN]
  C --> D[結合相手 NULL の行のみ残す]
  D --> E[出力 name]
```

結論から言うと、数字（`NOT EXISTS` の方が速い）は理にかなっています。さらに伸ばす余地はあります。

---

## まず結論：推奨クエリ

### 会社名が一意（`Company(name)` が UNIQUE or 実質一意）の場合

```sql
-- Company(name) が一意なら、Orders だけを相関で見るのが最短経路
SELECT s.name
FROM SalesPerson AS s
WHERE NOT EXISTS (
  SELECT 1
  FROM Orders AS o
  WHERE o.sales_id = s.sales_id
    AND o.com_id = (SELECT com_id FROM Company WHERE name = 'RED')
);

Runtime 310 ms
Beats 89.57%

```

*メリット*: `Company` への結合を実行計画上ほぼ「定数化」でき、`Orders` の `com_id, sales_id` でピンポイント検索に落ちやすい。`DISTINCT` も不要。

### 会社名が一意でない可能性がある場合（安全策）

```sql
WITH rc AS (
  SELECT com_id
  FROM Company
  WHERE name = 'RED'
)
SELECT s.name
FROM SalesPerson AS s
WHERE NOT EXISTS (
  SELECT 1
  FROM Orders AS o
  WHERE o.sales_id = s.sales_id
    AND o.com_id IN (SELECT com_id FROM rc)
);

Runtime 306 ms
Beats 94.84%

```

*ポイント*: `rc` は 16+ では基本「インライン化」されるため、実行計画上コスト増は最小。`LEFT JOIN ... IS NULL` よりも `NOT EXISTS` が安定して速いことが多いです（`DISTINCT` や重複排除が不要なため）。

---

## インデックス設計（ここが一番効きます）

1. ルックアップ用

```sql
CREATE INDEX IF NOT EXISTS ix_company_name ON Company(name);
```

2. 相関サブクエリを支える複合索引（順序が重要）

```sql
-- com_id で絞った後に sales_id を照合するアクセスに最適
CREATE INDEX IF NOT EXISTS ix_orders_com_sales ON Orders(com_id, sales_id);
```

> 代替（集約や DISTINCT を使う場合）として `Orders(sales_id, com_id)` も候補ですが、今回の述語順序（`com_id` ⇒ `sales_id`）なら上記が合致します。

3. PK は既定として存在（`SalesPerson(sales_id)`）

---

## なぜ速くなるのか

* `NOT EXISTS` は **最初の一致で即座に打ち切り**できる（アンチセミ結合）。
* `LEFT JOIN ... IS NULL` よりも **重複排除や広がり結合を避けられる**ので、`Orders` が大きいほど差が出やすい。
* `Company(name)` が一意なら **スカラサブクエリが定数化**され、`Orders` への探索キーがはっきりする。

---

## 実行計画で確認すべき点（`EXPLAIN (ANALYZE, BUFFERS)`）

* `Orders` 側が **Index Scan**（`ix_orders_com_sales`）になっているか
* `Company(name='RED')` が **Index Scan/Seek**（`ix_company_name`）になっているか
* 結合方法が **Anti Join (Nested Loop Anti / Hash Anti)** になっているか
* ループ毎に `Orders` を **再使用（パラメタライズド Index Scan）** できているか

---

## 追加のマイクロ最適化

* `WHERE` のリテラルは **完全一致**（`name = 'RED'`）。`ILIKE` などは避ける。
* 統計の更新：`ANALYZE Company; ANALYZE Orders;`（更新が多いなら `autovacuum` 設定も）
* 必要な列だけ参照（今回すでに最小）
* 結果順任意なら `ORDER BY` なし（そのままでOK）

---

## まとめ

* あなたの比較結果どおり、**`NOT EXISTS` が優位**。
* さらに **`Company(name)` と `Orders(com_id, sales_id)` の索引**を用意し、上記の「一意前提版」または「安全版」`NOT EXISTS` に差し替えると、**数十〜数百 ms 単位で短縮**が期待できます。
* 最後は `EXPLAIN (ANALYZE, BUFFERS)` で **Index Scan / Anti Join** になっていることを確認してください。

